In [36]:
def initalize_maze(maze):
    initalized_maze = []
    for row in maze:
        initalized_maze.append(row.split())
    
    initalized_board = initalized_maze.copy()
    for row in range(len(initalized_board)):
        for column in range(len(initalized_board[row])):
            if initalized_board[row][column] == "S":
                initalized_board[row][column] = "-"
    
    return initalized_maze, initalized_board

maze = ["S - - - - -",
        "- # # - | -",
        "- - | - | -",
        "- # # - # #",
        "- - | - - -",
        "| - | - | E"]

maze, board = initalize_maze(maze)

display_progress = False
display_board = False

In [37]:
from IPython.display import display, clear_output
import time
import matplotlib.pyplot as plt

def Display_Progress(n_generations=0, wait=0):
    if display_progress:
        if n_generations == 0:
            clear_output(wait=True)
            
            display("Generation " + str(agent.generation))
            
            plt.plot(agent.gen_rewards)
            plt.show()
            
            plt.plot(agent.gen_actions)
            plt.show()
            
            time.sleep(wait)
        elif (agent.generation % n_generations) == 0:
            clear_output(wait=True)
            
            display("Generation " + str(agent.generation))
            
            plt.plot(agent.gen_rewards)
            plt.show()
            
            plt.plot(agent.gen_actions)
            plt.show()
            
            time.sleep(wait)

def Update_Board():
    if display_board:
        clear_output(wait=True)
        
        display("Generation " + str(agent.generation))
        display("Terminal Point " + str(agent.env.Terminal_Point()))
        
        for row in range(len(board)):
            text = ""
            for column in range(len(board[row])):
                text = text + board[row][column] + "  "
            display(text)
            
        x, y = player.XY()
        tile = x + (len(maze[0]) * y)
        display(agent.Q[tile])

In [38]:
class Player:
    def __init__(self, icon="o", blank="-", slow=False, speed_limit=1) -> None:
        self.icon = icon
        self.blank = blank
        self.slow = slow
        self.speed_limit = speed_limit
        self.time = 0
        
        for row in range(len(maze)):
            for column in range(len(maze[row])):
                if maze[row][column] == "S":
                    board[row][column] = self.icon
        
    def XY(self):
        x = 0
        y = 1
        
        for row in range(len(board)):
            for column in range(len(board[row])):
                if board[row][column] == self.icon:
                    x = column
                    y = row
        return x, y
    
    def State(self):
        x, y = self.XY()
        return x + (len(maze[0]) * y)
    
    def Terminal_Point(self):
        x, y = self.XY()
        return (x == 5 and y == 5)
    
    def Reward(self):
        x, y = self.XY()
        reward = 0
        if not maze[y][x] == "E":
            reward = -1
        return reward
    
    def Spawn(self):
        global board
        x, y = self.XY()
        board[y][x] = "E"

        Display_Progress(n_generations=20, wait=5)

        for row in range(len(maze)):
            for column in range(len(maze[row])):
                if maze[row][column] == "S":
                    board[row][column] = self.icon
                    
        self.time = 0

    def Right(self):
        global board
        x, y = self.XY()
        
        board[y][x] = self.icon
        
        Update_Board()
        if self.slow:
            time.sleep(self.speed_limit)
            
        if x+1 < len(maze[y]):
            if maze[y][x+1] == self.blank or maze[y][x+1] == "E":
                board[y][x] = self.blank
                board[y][x+1] = self.icon
        
        self.time += 1
                
    def Left(self):
        global board
        x, y = self.XY()
        
        board[y][x] = self.icon
        
        Update_Board()
        if self.slow:
            time.sleep(self.speed_limit)
        
        if x-1 >= 0:
            if maze[y][x-1] == self.blank or maze[y][x-1] == "E":
                board[y][x] = self.blank
                board[y][x-1] = self.icon
                
        self.time += 1
                
    def Down(self):
        global board
        x, y = self.XY()
        
        board[y][x] = self.icon
        
        Update_Board()
        if self.slow:
            time.sleep(self.speed_limit)

        if y+1 < len(maze):
            if maze[y+1][x] == self.blank or maze[y+1][x] == "E":
                board[y][x] = self.blank
                board[y+1][x] = self.icon
                
        self.time += 1
                
    def Up(self):
        global board
        x, y = self.XY()
        
        board[y][x] = self.icon
        
        Update_Board()
        if self.slow:
            time.sleep(self.speed_limit)
        
        if y-1 >= 0:
            if maze[y-1][x] == self.blank or maze[y-1][x] == "E":
                board[y][x] = self.blank
                board[y-1][x] = self.icon
                
        self.time += 1

In [39]:
from ReinforcementLearning import Action, Enviorment, QLearning, State

player = Player(speed_limit=0.3)

states = []
states.append(State("Location", 0, player.State, n_discrete=(len(maze))**2 ))

actions = [Action("Right", 0, player.Right), Action("Left", 1, player.Left), Action("Down", 2, player.Down), Action("Up", 3, player.Up)]

enviorment = Enviorment(player.Reward, player.Spawn, player.Terminal_Point, states, actions)

In [40]:
display_progress = True

agent = QLearning(enviorment, save_rewards=True)
agent.Training_Mode(max_iterations=200) 

display_progress = False
display_board = True
player.slow = True

agent.Inference_Mode()

print("Done")

'Generation 200'

'Terminal Point False'

'-  -  -  -  -  -  '

'-  #  #  -  |  -  '

'-  -  |  -  |  -  '

'-  #  #  -  #  #  '

'-  -  |  -  -  o  '

'|  -  |  -  |  E  '

[-1.99, -2.9701, -1.0, -1.99]

Done
